In [ ]:
!pip install transformers datasets tokenizers
!wget  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -qq cornell_movie_dialogs_corpus.zip #-qq flag indicate quiet mode where it will suppress all the output except for error message
!rm cornell_movie_dialogs_corpus.zip
!mkdir datasets
!mv cornell\ movie-dialogs\ corpus/movie_conversations.txt ./datasets
!mv cornell\ movie-dialogs\ corpus/movie_lines.txt ./datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
import os
import torch, re, random
import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import numpy as np, math, itertools
import tqdm
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import Adam
from pathlib import Path

## Tokenization:


In [ ]:
from collections import defaultdict
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')

corpus=[
     "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
#@ Calculating the frequency of each word:
word_freqs=defaultdict(int)
for text in corpus:
  words_with_offsets=tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
  new_words=[word for word, offset in words_with_offsets]
  print(new_words)
  for word in new_words:
    word_freqs[word]+=1

print(f'\nfinal word freq"{word_freqs}')

['This', 'is', 'the', 'Hugging', 'Face', 'Course', '.']
['This', 'chapter', 'is', 'about', 'tokenization', '.']
['This', 'section', 'shows', 'several', 'tokenizer', 'algorithms', '.']
['Hopefully', ',', 'you', 'will', 'be', 'able', 'to', 'understand', 'how', 'they', 'are', 'trained', 'and', 'generate', 'tokens', '.']

final word freq"defaultdict(<class 'int'>, {'This': 3, 'is': 2, 'the': 1, 'Hugging': 1, 'Face': 1, 'Course': 1, '.': 4, 'chapter': 1, 'about': 1, 'tokenization': 1, 'section': 1, 'shows': 1, 'several': 1, 'tokenizer': 1, 'algorithms': 1, 'Hopefully': 1, ',': 1, 'you': 1, 'will': 1, 'be': 1, 'able': 1, 'to': 1, 'understand': 1, 'how': 1, 'they': 1, 'are': 1, 'trained': 1, 'and': 1, 'generate': 1, 'tokens': 1})


In [ ]:
#@ Splitting all the words in alphabet:
alphabet=[]
for word in word_freqs.keys():
  if word[0] not in alphabet:
    alphabet.append(word[0])

  for letter in word[1:]:
    if f"##{letter}" not in alphabet:
      alphabet.append(f'##{letter}')

alphabet.sort()
print(f'All alphabet: {alphabet}')

All alphabet: ['##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y']


In [ ]:
#@ adding special tokens:
vocab=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]'] + alphabet.copy()
print(vocab)

['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y']


In [ ]:
splits={word: [c if i==0 else f'##{c}' for i,c in enumerate(word)] for word in word_freqs.keys()}
print(f'\n Splitted words: {splits}')


 Splitted words: {'This': ['T', '##h', '##i', '##s'], 'is': ['i', '##s'], 'the': ['t', '##h', '##e'], 'Hugging': ['H', '##u', '##g', '##g', '##i', '##n', '##g'], 'Face': ['F', '##a', '##c', '##e'], 'Course': ['C', '##o', '##u', '##r', '##s', '##e'], '.': ['.'], 'chapter': ['c', '##h', '##a', '##p', '##t', '##e', '##r'], 'about': ['a', '##b', '##o', '##u', '##t'], 'tokenization': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##a', '##t', '##i', '##o', '##n'], 'section': ['s', '##e', '##c', '##t', '##i', '##o', '##n'], 'shows': ['s', '##h', '##o', '##w', '##s'], 'several': ['s', '##e', '##v', '##e', '##r', '##a', '##l'], 'tokenizer': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##e', '##r'], 'algorithms': ['a', '##l', '##g', '##o', '##r', '##i', '##t', '##h', '##m', '##s'], 'Hopefully': ['H', '##o', '##p', '##e', '##f', '##u', '##l', '##l', '##y'], ',': [','], 'you': ['y', '##o', '##u'], 'will': ['w', '##i', '##l', '##l'], 'be': ['b', '##e'], 'able': ['a', '##b', '##l', '##e'], '

In [ ]:
#@ computing scores for merging:

def compute_pair_scores(splits):
  letter_freqs=defaultdict(int)
  pair_freqs=defaultdict(int)

  for word, freq in word_freqs.items():
    split=splits[word]
    if len(split) == 1:
      letter_freqs[split[0]] += freq
      continue

    for i in range(len(split) -1):
      pair=(split[i], split[i+1])
      letter_freqs[split[i]] += freq
      pair_freqs[pair] += freq
    letter_freqs[split[-1]] += freq

  scores= {
      pair: freq/(letter_freqs[pair[0]] * letter_freqs[pair[1]])

      for pair, freq in pair_freqs.items()
  }
  return scores

pair_scores=compute_pair_scores(splits)
print(f'Scores for each pair: {pair_scores}')

Scores for each pair: {('T', '##h'): 0.125, ('##h', '##i'): 0.03409090909090909, ('##i', '##s'): 0.02727272727272727, ('i', '##s'): 0.1, ('t', '##h'): 0.03571428571428571, ('##h', '##e'): 0.011904761904761904, ('H', '##u'): 0.1, ('##u', '##g'): 0.05, ('##g', '##g'): 0.0625, ('##g', '##i'): 0.022727272727272728, ('##i', '##n'): 0.01652892561983471, ('##n', '##g'): 0.022727272727272728, ('F', '##a'): 0.14285714285714285, ('##a', '##c'): 0.07142857142857142, ('##c', '##e'): 0.023809523809523808, ('C', '##o'): 0.07692307692307693, ('##o', '##u'): 0.046153846153846156, ('##u', '##r'): 0.022222222222222223, ('##r', '##s'): 0.022222222222222223, ('##s', '##e'): 0.004761904761904762, ('c', '##h'): 0.125, ('##h', '##a'): 0.017857142857142856, ('##a', '##p'): 0.07142857142857142, ('##p', '##t'): 0.07142857142857142, ('##t', '##e'): 0.013605442176870748, ('##e', '##r'): 0.026455026455026454, ('a', '##b'): 0.2, ('##b', '##o'): 0.038461538461538464, ('##u', '##t'): 0.02857142857142857, ('t', '##o')

In [ ]:
#@ Finding pair with the best score:
best_pair=''
max_score=None

for pair, score in pair_scores.items():
  if max_score is None or max_score < score:
    best_pair=pair
    max_score=score

print(best_pair, max_score)
vocab.append('ab')

#@ merging the pair:
def merge_pair(a, b, splits):
  for word in word_freqs:
    split=splits[word]
    if len(split)==1:
      continue
    i=0
    while i < len(split)-1:
      if split[i] == a and split[i+1]==b:
        merge= a + b[2:] if b.startswith('##') else a + b
      else:
        i+= 1
    splits[word] = split
  return splits

splits=merge_pair('a', '##b', splits)
print(splits['about'])


('a', '##b') 0.2


In [ ]:
#@ Keep looping to merge pair:
vocab_size=70

while len(vocab) < vocab_size:
  scores=compute_pair_scores(splits)
  best_pair, max_score="", None
  for pair, score in scores.items():
    if max_score is None or max_score < score:
      best_pair=pair
      max_score=score
  splits=merge_pair(*best_pair, splits)
  new_token=(
      best_pair[0] + best_pair[1][2:]
      if best_pair[1].startswith('##')
      else best_pair[0] + best_pair[1]
  )
  vocab.append(new_token)

print(f'Final Vocab: {vocab}')




## Pre-processing:


In [ ]:
class BERTDataset(Dataset):
  def __init__(self, data_pair, tokenizer, seq_len=64):
    self.tokenizer=tokenizer
    self.seq_len=seq_len
    self.corpus_lines=len(data_pair)
    self.lines=data_pair

  def __len__(self):
    return self.corpus_lines

  def __getitem__(self, item):
    #getting random sentence pair(saved as is_next_label)
    t1, t2, is_next_label=self.get_sent(item)

    #replacing random words in sentence with mask/random words:
    t1_random, t1_label=self.random_word(t1)
    t2_random, t2_label=self.random_word(t2)

    #adding special tokens:
    t1=[self.tokenizer.vocab['[CLS]']] + t1_random + [self.tokenizer.vocab['[SEP]']]
    t2=t2_random + [self.tokenizer.vocab['[SEP]']]
    t1_label=[self.tokenizer.vocab['[PAD]']] + t1_label + [self.tokenizer.vocab['[PAD]']]
    t2_label +=[self.tokenizer.vocab['[PAD]']]

    #combining sentence one and two as one input and adding PAD tokens
    #to make sentence same length as seq_len
    segment_label=([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
    bert_input=(t1+t2)[:self.seq_len]
    bert_label=(t1_label + t2_label)[:self.seq_len]
    padding=[self.tokenizer.vocab['[PAD]'] for _ in range(self.seq_len - len(bert_input))]
    bert_input.extend(padding), bert_label.extend(padding), segment_label.extend(padding)

    output={
        "bert_input": bert_input,
        "bert_label":bert_label,
        "segment_label": segment_label,
        "is_next": is_next_label
    }

    return {key: torch.tensor(value) for key, value in output.items()}

  def random_word(self, sentence):
    tokens=sentence.split()
    output=[] #stores modified tokens id
    output_label=[] # stores original token id

    # according to the bert paper, 15% of the tokens are generally replaced:
    for  i, token in enumerate(tokens):
      prob=random.random() #generate random prob between 0 to 1

      #removing special tokens: [CLS] and [SEP]
      token_id=self.tokenizer(token)['input_ids'][1:-1]

      # 15% token position at random:
      if prob < 0.15:
        prob /= 0.15

        # 80% masking in the selected position:
        if prob < .8:
          for i in range(len(token_id)):
            output.append(self.tokenizer.vocab['[MASK]'])

        # 10% change to random token:
        elif prob < 0.9:
          for i in range(len(token_id)):
            output.append(random.randrange(len(self.tokenizer.vocab)))

        # 10% of the current token:
        else:
          output.append(token_id)

        output_label.append(token_id)

      else:
        output.append(token_id)
        for i in range(len(token_id)):
          output_label.append(0)

    # flattening is done:
    output=list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output]))
    output_label=list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output_label]))
    assert len(output) == len(output_label)
    return output, output_label

  def get_sent(self, index):
    """return random sentence pair"""
    t1, t2=self.get_corpus_line(index)

    #for next sentence prediction:
    if random.random() > 0.5:
      return t1, t2, 1
    else:
      return t1, self.get_random_line(), 0


  def get_corpus_line(self, item):
    """return sentence pair"""
    return self.lines[item][0], self.lines[item][1]

  def get_random_line(self):
    """return random single sentence"""
    return self.lines[random.randrange(len(self.lines))][1]


### Modeling:

In [ ]:
class PositionalEncoding(torch.nn.Module):
  def __init__(self, d_model, max_len=128):
    super().__init__()

    pe=torch.zeros(max_len, d_model).float()
    pe.require_grad=False


    for pos in range(max_len):
      for i in range(0, d_model, 2):
        pe[pos, i]=math.sin(pos/(10000**((2*i)/d_model)))
        pos[pos, i+1]=math.cos(pos/(10000**((2*(i+1))/d_model)))

    #including the batch size:
    self.pe=pe.unsqueeze(0)

  def forward(self, x):
    return self.pe

In [ ]:
class BERTEmbedding(torch.nn.Module):
  """
  BERT Emmbedding consisted of thrree features:
  1. token embeddings: normal embeddings matrix,
  2. PositionalEmbeddings: addition of position of tokens
  3. Segment Embeddings: add sentence segment info
  """
  def __init__(self, vocab_size, embed_size, seq_len=4, dropout=0.1):
    super().__init__()
    self.embed_size=embed_size
    self.token=torch.nn.Embedding(vocab_size, embed_size, padding_idx=0)
    self.segment=torch.nn.Embedding(3, embed_size, padding_idx=0)
    self.position=PositionalEncoding(d_model=embed_size, max_len=seq_len)
    self.dropout=torch.nn.dropout(p=dropout)

  def forward(self, sequence, segment_label):
    x=self.token(sequence) + self.position(sequence) + self.segment(segment_label)
    return self.dropout(x)

In [ ]:
class MultiHeadedAttention(torch.nn.Module):
  def __init__(self, heads, d_model, dropout=0.1):
    super(MultiHeadedAttention, self).__init__()
    assert d_model % heads == 0
    self.d_k=d_model // heads #dimension of each head
    self.heads=heads
    self.dropout=torch.nn.Dropout(dropout)

    self.query=torch.nn.Linear(d_model, d_model)
    self.key=torch.nn.Linear(d_model, d_model)
    self.value=torch.nn.Linear(d_model, d_model)
    self.output_linear=torch.nn.Linear(d_model, d_model)

  def forward(self, query, value, key, mask):
    query=self.query(query)
    key=self.key(key)
    value=self.value(value)

    query=query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
    key=key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
    value=value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)

    scores=torch.matmul(query, key.permute(0, 1, 3,2))//math.sqrt(query.size(-1))

    #masking
    scores=scores.masked_fill(mask==0, -1e9)

    weights=F.softmax(scores, dim=-1)
    weights=self.dropout(weights)

    context=torch.matmul(weights, value)

    context=context.permute(0,2,1,3)

    return self.output_linear(context)

### **Mathematical Definition of GELU**
The **Gaussian Error Linear Unit (GELU)** activation function is defined as:

$$
\text{GELU}(x) = x \cdot \Phi(x)
$$

where \( \Phi(x) \) is the **Cumulative Distribution Function (CDF)** of the standard normal distribution:

$$
\Phi(x) = \frac{1}{2} \left( 1 + \tanh \left( \sqrt{\frac{2}{\pi}} \left( x + 0.044715 x^3 \right) \right) \right)
$$

### **Why is GELU Important?**
- Unlike **ReLU**, which clips negative values to zero, GELU smoothly scales negative values, making it more expressive.
- Unlike **Leaky ReLU**, GELU is **nonlinear** in a way that aligns with **probabilistic modeling**.


In [ ]:
class FeedForward(torch.nn.Module):
  def __init__(self, d_model, middle_dim=2048, dropout=0.1):
    super(FeedForward, self).__init__()

    self.fc1=torch.nn.Linear(d_model, middle_dim)
    self.fc2=torch.nn.Linear(middle_dim, d_model)
    self.dropout=torch.nn.Dropout(dropout)
    self.activation=torch.nn.GELU()

  def forward(self, x):
    out=self.activation(self.fc1(x))
    out=self.fc2(self.dropout(out))
    return out

In [ ]:
class Encoder(torch.nn.Module):
  def __init__(self, d_model=768, heads=12, feed_forward_hidden=768 * 4, dropout=0.1):
    super(Encoder, self).__init__()
    self.layernorm=torch.nn.LayerNorm(d_model)
    self.self_multihead=MultiHeadedAttention(heads, d_model)
    self.feed_forward=FeedForward(d_model, middle_dim=feed_forward_hidden)
    self.dropout=torch.nn.Dropout(dropout)


  def forward(self, embeddings, mask):
    interacted=self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
    #residual error:
    interacted=self.layernorm(interacted + embeddings)

    #bottleneck:
    feed_forward_out=self.dropout(self.feed_forward(interacted))
    encoded=self.layernorm(feed_forward_out + interacted)
    return encoded


In [ ]:
class BERT(torch.nn.Module):
  def __init__(self, vocab_size, d_model=768, n_layers=12, heads=12, dropout=0.1):
    super(BERT, self).__init__()
    self.d_model=d_model
    self.n_layers=n_layers #encoder
    self.heads=heads
    self.ff_hidden=d_model * 4 # specified in paper
    self.embeddings=BERTEmbedding(vocab_size=vocab_size, embed_size=d_model)

    self.encoder_blocks=torch.nn.ModuleList(
        [Encoder(d_model, heads, d_model*4, dropout) for _ in range(n_layers)]
    )

  def forward(self, x, segment_info):
    #attention masking for padded tokens
    # (batch_size, 1, seq_len, seq_len)
    mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsequeeze(1)

    #embedding the indexed sequence to sequence of vectors:
    x=self.embeddings(x, segment_info)

    #for running over multiple transformer block:
    for encoder in self.encoder_blocks:
      x=encoder.forward(x, mask)
    return x



In [ ]:
class NextSentencePrediction(torch.nn.Module):
  """
  2-class classification model: 'is_next', 'is_not_next'
  """
  def __init__(self, hidden):
    """
    hidden: BERT Model output size
    """
    super().__init__()
    self.linear=torch.nn.Linear(hidden, 2)
    self.softmax=torch.nn.LogSoftmax(dim=-1)

  def forward(self, x):
    return self.softmax(self.linear(x[:, 0])) #'[CLS]' which represents the entire sequence


In [ ]:
class MaskedLangaugeModel(torch.nn.Module):
  """
  predicting original token from masked input sequence: n-class classification
  n-class:vocab_size
  """
  def __init__(self, hidden, vocab_size):
    """
    hidden: output size
    vocab_size=total unique words
    """
    super().__init__()
    self.linear=torch.nn.Linear(hidden, vocab_size)
    self.softmax-torch.nn.LogSoftmax(dim=-1)

  def forward(self, x):
    return self.softmax(self.linear(x))

In [ ]:
class BERTLM(torch.nn.Module):
  """
  BERT Language: NSP + MLM
  """
  def __init__(self, bert:BERT, vocab_size):
    """
    bert: BERT model which should be trained
    """
    super().__init__()
    self.bert=bert
    self.next_sentence=NextSentencePrediction(self.bert.d_model)
    self.mask_lm=MaskedLanguageModel(self.bert.d_model, vocab_size)

  def forward(self, x, segment_label):
    x= self.bert(x, segment_label)
    return self.next_sentence(x), self.mask_lm(x)